In [157]:
import torch as t
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim


In [158]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [159]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [160]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [161]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [162]:
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [163]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder .transform(y_test)

In [164]:
# now change thses array in pytorch array
x_train_tensor = t.from_numpy(x_train).float()
x_test_tensor = t.from_numpy(x_test).float()
y_train_tensor = t.from_numpy(y_train).float()
y_test_tensor = t.from_numpy(y_test).float()
# x_train_tensor = x_train_tensor 

In [165]:
# defining the model
class MySimpleNN(nn.Module):
    def __init__(self,num_feature):
        super().__init__()
        self.linear = nn.Linear(num_feature,1)
        self.sigmoid = nn.Sigmoid()
        # self.weight = t.rand(X.shape[1],1,dtype=t.float64,requires_grad=True)
        # self.bias = t.zeros(1,dtype=t.float64,requires_grad=True)
    def forward(self,X):
        out = self.linear(X)
        out = self.sigmoid(out)
        # z = t.matmul(X,self.weight)+self.bias
        # y_pred = t.sigmoid(z)
        return out
    

In [166]:
loss = nn.BCELoss()
type(loss)

torch.nn.modules.loss.BCELoss

In [167]:
x_train_tensor.shape

torch.Size([455, 30])

In [168]:
# parameter
learning_rate = 0.1
epochs = 10
model = MySimpleNN(x_train_tensor.shape[1])

In [169]:
# model
print(x_train_tensor.dtype),print(model.linear.weight.dtype)  # Or other layers of your model
y_train_tensor = y_train_tensor.reshape(-1,1)

torch.float32
torch.float32


In [171]:
# so now we want to use built-in loss function plus optimizer
# pytorch has well known optimizer class is known as optim
optimizer = optim.SGD(model.parameters(), lr=0.01)  # SGD with learning rate 0.01

# define loop
for epoch in range(epochs):
# 2) forward pass
    y_pred = model.forward(x_train_tensor)
# 3) caculate loss
    los = loss(y_pred,y_train_tensor)
# 3.1) it is best practice to zero all grad before backward
    optimizer.zero_grad()
# 4) backward pass
    los.backward()
# 5) update parameter
    optimizer.step()
    print(los)

tensor(0.2588, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2577, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2567, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2556, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2546, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2536, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2526, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2516, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2506, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2496, grad_fn=<BinaryCrossEntropyBackward0>)


In [155]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[ 0.0508,  0.2262,  0.1309,  0.0935,  0.0330,  0.2449,  0.2827,  0.0734,
         -0.0486, -0.0887,  0.2136, -0.0112,  0.0799,  0.2430, -0.0898, -0.0522,
          0.0273, -0.0387,  0.0117, -0.1342,  0.0176,  0.2387,  0.2954,  0.1329,
          0.1889,  0.2121,  0.0549,  0.2840,  0.1028,  0.0436]],
       requires_grad=True)
Parameter containing:
tensor([-0.2185], requires_grad=True)
